<a href="https://colab.research.google.com/github/mfbalin/dgl-1/blob/gb_doc_fixes/notebooks/stochastic_training/multigpu_node_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
torch.__version__

'2.1.0+cu121'

In [ ]:
asdasd